# प्राथमिकता सदस्य मिडलवेयर सहित होटल बुकिङ

यो नोटबुकले माइक्रोसफ्ट एजेन्ट फ्रेमवर्क प्रयोग गरी **फन्क्सन-आधारित मिडलवेयर** प्रदर्शन गर्दछ। हामी सर्तमूलक वर्कफ्लो उदाहरणमा एउटा मिडलवेयर तह थप्छौं जसले प्राथमिकता सदस्यहरूलाई विशेष सुबिधा दिन्छ।

## तपाईंले के सिक्नुहुनेछ:
1. **फन्क्सन-आधारित मिडलवेयर**: फन्क्सन परिणामहरू रोक्ने र परिमार्जन गर्ने
2. **सन्दर्भ पहुँच**: कार्यान्वयन पछि `context.result` पढ्ने र परिमार्जन गर्ने
3. **व्यवसायिक तर्क कार्यान्वयन**: प्राथमिकता सदस्य लाभहरू
4. **परिणाम ओभरराइड**: प्रयोगकर्ता स्थितिमा आधारित फन्क्सन नतिजा परिवर्तन गर्ने
5. ** एउटै वर्कफ्लो, फरक नतिजाहरू**: मिडलवेयर-प्रेरित व्यवहार परिवर्तनहरू

## मिडलवेयर सहित वर्कफ्लो वास्तुकला:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## सर्तमूलक वर्कफ्लोबाट मुख्य फरक:

**मिडलवेयर बिना** (14-conditional-workflow.ipynb):
- पेरिसमा कोठा छैन → alternative_agent तर्फ मार्ग

**मिडलवेयर सहित** (यो नोटबुक):
- सामान्य प्रयोगकर्ता + पेरिस → कोठा छैन → alternative_agent तर्फ मार्ग
- प्राथमिकता प्रयोगकर्ता + पेरिस → 🌟 मिडलवेयरले ओभरराइड गर्यो! → उपलब्ध → booking_agent तर्फ मार्ग

## पूर्वशर्तहरू:
- माइक्रोसफ्ट एजेन्ट फ्रेमवर्क स्थापना गरिएको
- सर्तमूलक वर्कफ्लोहरूको समझ (हेर्नुहोस् 14-conditional-workflow.ipynb)
- GitHub टोकन वा OpenAI API कुञ्जी
- मिडलवेयर ढाँचाहरूको आधारभूत बुझाइ


In [1]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Step 1: संरचित आउटपुटका लागि Pydantic मोडेलहरू परिभाषित गर्नुहोस्

यी मोडेलहरूले एजेन्टहरूले फिर्ता गर्ने **स्कीमा** परिभाषित गर्दछन्। हामीले मिडलवेयरले उपलब्धता नतिजा परिवर्तन गर्दा ट्र्याक गर्नका लागि `priority_override` फिल्ड थपेका छौं।


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Step 2: प्राथमिक सदस्यहरूको डाटाबेस परिभाषित गर्नुहोस्

यस डेमोको लागि, हामी प्राथमिक सदस्यता डाटाबेसको सिमुलेशन गर्नेछौं। उत्पादनमा, यो वास्तविक डाटाबेस वा API लाई सोध्नेछ।

**प्राथमिक सदस्यहरू:**
- `alice@example.com` - VIP सदस्य
- `bob@example.com` - प्रिमियम सदस्य  
- `priority_user` - टेस्ट खाता


In [3]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## Step 3: होटल बुकिङ उपकरण तयार गर्नुहोस्

सशर्त वर्कफ्लो जस्तै नै, तर अब यसलाई मिडलवेयरले रोक्नेछ!


In [4]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Step 4: 🌟 प्राथमिकता जाँच मिडलवेयर सिर्जना गर्नुहोस् (मुख्य विशेषता!)

यो नोटबुकको **मुख्य कार्यक्षमता** हो। मिडलवेयरले:

1. होटल_बुकिंग फंक्शन कललाई **इन्टरसेप्ट गर्छ**
2. `next(context)` कल गरेर फंक्शन सामान्य रूपमा **कार्यान्वयन गर्छ**
3. `context.result` मा नतिजा **निरीक्षण गर्छ**
4. प्रयोगकर्ता प्राथमिकता भएमा र कोठा उपलब्ध नभएमा नतिजा **ओभरराइड गर्छ**
5. संशोधित नतिजा एजेन्टलाई **फिर्ता गर्छ**

**मुख्य ढाँचा:**  
```python
async def my_middleware(context, next):
    await next(context)  # प्रकार्य कार्यान्वयन गर्नुहोस्
    # अब context.result मा प्रकार्यको नतिजा छ
    if some_condition:
        context.result = new_value  # ओभरराइड!
```


In [5]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## Step 5: रुटिङका लागि अवस्था कार्यहरू परिभाषित गर्नुहोस्

शर्तात्मक कार्यप्रवाह जस्तै उस्तै अवस्था कार्यहरूले संरचित आउटपुटको निरीक्षण गरी रुटिङ निर्धारण गर्छन्।


In [6]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## Step 6: अनुकूलित प्रदर्शन एक्ज़िक्यूटर सिर्जना गर्नुहोस्

पहिलेको जस्तो एक्ज़िक्यूटरै हो - अन्तिम वर्कफ्लो आउटपुट प्रदर्शन गर्दछ।


In [7]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## Step 7: वातावरण चरहरू लोड गर्नुहोस्

LLM क्लाएन्ट (GitHub मोडेलहरू वा OpenAI) कन्फिगर गर्नुहोस्।


In [8]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## Step ८: मिडलवेयरसँग AI एजेन्टहरू सिर्जना गर्नुहोस्

**मुख्य भिन्नता:** availability_agent सिर्जना गर्दा, हामीले `middleware` प्यारामिटर पास गर्छौं!

यसरी हामी priority_check_middleware लाई एजेन्टको फंक्शन कल पाइलाइनमा इन्जेक्ट गर्छौं।


In [9]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Step 9: वर्कफ्लो निर्माण गर्नुहोस्

अघिल्लो जस्तै उही वर्कफ्लो संरचना - उपलब्धतामा आधारित सर्तात्मक राउटिङ।


In [10]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## Step 10: Test Case 1 - पेरिसमा नियमित प्रयोगकर्ता (ओभरराइड नगर्ने)

एउटा नियमित प्रयोगकर्ताले पेरिस बुक गर्न प्रयास गर्दछ → कोठाहरु उपलब्ध छैनन् → वैकल्पिक_agent तर्फ मार्गनिर्देशन गर्छ।


In [11]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## Step 11: Test Case 2 - 🌟 प्राथमिकता प्रयोगकर्ता पेरिसमा (ओभरराइडसँग!)

एउटा प्राथमिकता सदस्यले पेरिस बुक गर्न खोज्छ → सुरूमा कोठा छैन → 🌟 मिडलवेयरले ओभरराइड गर्यो! → booking_agent मा रूट गरिन्छ

**यो मिडलवेयरको शक्तिको मुख्य प्रदर्शन हो!**


In [12]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## Step 12: परीक्षण केस 3 - स्टकहोममा प्राथमिकता प्रयोगकर्ता (पहिले नै उपलब्ध)

प्राथमिकता प्रयोगकर्ताले स्टकहोम प्रयास गर्छ → कोठाहरू उपलब्ध → कुनै ओभरराइड आवश्यक छैन → booking_agent मा रुट हुन्छ

यसले देखाउँछ कि मिडलवेयर केवल आवश्यकता पर्दा मात्र कार्य गर्छ!


In [13]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## मुख्य सन्देश र मिडलवेयर अवधारणाहरू

### ✅ तपाइँले सिक्नुभएको कुरा:

#### **१. फङ्क्शन-आधारित मिडलवेयर ढाँचा**

मिडलवेयरले फङ्क्शन कलहरू एक साधारण असिंक्रोनस फङ्क्शन प्रयोग गरेर रोक्छ:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # कार्यसम्पादन अघि
    print("Intercepting...")
    
    # कार्यसम्पादन गर्नुहोस्
    await next(context)
    
    # कार्यसम्पादन पछि - नतिजा जाँच गर्नुहोस्
    if context.result:
        # आवश्यकता अनुसार नतिजा परिवर्तन गर्नुहोस्
        context.result = modified_value
```

#### **२. सन्दर्भ पहुँच र परिणाम अधिलेखन**

- `context.function` - कल भइरहेको फङ्क्शनमा पहुँच
- `context.arguments` - फङ्क्शनका आर्गुमेन्टहरू पढ्नुहोस्
- `context.kwargs` - थप प्यारामिटरहरूमा पहुँच
- `await next(context)` - फङ्क्शनलाई कार्यान्वयन गर्नुहोस्
- `context.result` - फङ्क्शनको आउटपुट पढ्न/परिमार्जन गर्न

#### **३. व्यावसायिक तर्क कार्यान्वयन**

हाम्रो मिडलवेयर प्राथमिकता सदस्य सुविधाहरू लागू गर्छ:
- **सामान्य प्रयोगकर्ता**: कुनै परिवर्तन छैन, मानक कार्यप्रवाह
- **प्राथमिकता प्रयोगकर्ता**: "उपलब्धता छैन" लाई "उपलब्ध" मा अधिलेखन गर्नुहोस्
- **सशर्त तर्क**: केवल आवश्यक परिप्रेक्षमा अधिलेखन

#### **४. उही कार्यप्रवाह, फरक नतिजाहरू**

मिडलवेयरको शक्ति:
- ✅ कार्यप्रवाह संरचनामा कुनै परिवर्तन छैन
- ✅ टुल फङ्क्शनमा कुनै परिवर्तन छैन
- ✅ सशर्त राउटिङ तर्कमा कुनै परिवर्तन छैन
- ✅ केवल मिडलवेयर → फरक व्यवहार!

### 🚀 वास्तविक संसारका अनुप्रयोगहरू:

1. **भिएआईपी/प्रीमियम सुविधाहरू**
   - प्रीमियम प्रयोगकर्ताहरूका लागि दर सीमा अधिलेखन
   - स्रोतहरूमा प्राथमिकता पहुँच प्रदान गर्नुहोस्
   - प्रीमियम सुविधाहरू गतिशील रूपमा अनलक गर्नुहोस्

2. **ए/बी परीक्षण**
   - प्रयोगकर्ताहरूलाई विभिन्न कार्यान्वयनमा मार्गनिर्देशन गर्नुहोस्
   - विशिष्ट प्रयोगकर्तासँग नयाँ सुविधाहरू परीक्षण गर्नुहोस्
   - सुविधाहरू क्रमशः रोलआउट गर्नुहोस्

3. **सुरक्षा र अनुपालन**
   - फङ्क्शन कलहरूको अडिट गर्नुहोस्
   - संवेदनशील अपरेशनहरू अवरुद्ध गर्नुहोस्
   - व्यावसायिक नियमहरू लागू गर्नुहोस्

4. **प्रदर्शन अनुकूलन**
   - विशिष्ट प्रयोगकर्ताहरूका लागि परिणामहरू क्यास गर्नुहोस्
   - जहिले संभव हो महँगो अपरेशनहरू छोड्नुहोस्
   - स्रोतको गतिशील आवंटन

5. **त्रुटि सम्हाल्ने र पुन: प्रयास**
   - त्रुटिहरूलाई समात्नुहोस् र सजिलै सम्हाल्नुहोस्
   - पुनः प्रयास तर्क लागू गर्नुहोस्
   - वैकल्पिक कार्यान्वयनमा फिर्ता जानुहोस्

6. **लगिङ र अनुगमन**
   - फङ्क्शन कार्यान्वयन समय ट्र्याक गर्नुहोस्
   - प्यारामिटरहरू र परिणामहरू लग गर्नुहोस्
   - उपयोग ढाँचाहरू अनुगमन गर्नुहोस्

### 🔑 डेकोरेटर्स भन्दा मुख्य भिन्नताहरू:

| विशेषता | डेकोरेटर | मिडलवेयर |
|---------|-----------|------------|
| **परिधि** | एकल फङ्क्शन | एजेन्टका सबै फङ्क्शनहरू |
| **लचकता** | परिभाषामा निश्चित | रनटाइममा गतिशील |
| **सन्दर्भ** | सीमित | पूर्ण एजेन्ट सन्दर्भ |
| **संरचना** | धेरै डेकोरेटरहरू | मिडलवेयर पाइपलाइन |
| **एजेन्ट-जानकार** | होइन | हो (एजेन्ट राज्यमा पहुँच) |

### 📚 कहिले मिडलवेयर प्रयोग गर्ने:

✅ **मिडलवेयर प्रयोग गर्नुहोस् जब:**
- तपाइँले प्रयोगकर्ता/सत्र स्थितिमा आधारित व्यवहार परिमार्जन गर्न आवश्यक छ
- तपाइँले धेरै फङ्क्शनमा तर्क लागू गर्न चाहानुहुन्छ
- तपाइँलाई एजेन्ट-स्तर सन्दर्भमा पहुँच चाहिन्छ
- तपाइँ क्रस-कटिङ विषयहरू (लगिङ, प्रमाणीकरण आदि) लागू गर्दै हुनुहुन्छ

❌ **मिडलवेयर प्रयोग नगर्नुहोस् जब:**
- सरल इनपुट मान्यकरण (पाइडान्टिक प्रयोग गर्नुहोस्)
- फङ्क्शन-विशिष्ट तर्क (फङ्क्शन भित्र राख्नुहोस्)
- एक पटकको परिवर्तनहरू (सिधै फङ्क्शन परिवर्तन गर्नुहोस्)

### 🎓 उन्नत ढाँचाहरू:

```python
# बहु मध्यवर्ती (कार्य सम्पन्न हुने क्रममा महत्वपुर्ण!)
middleware=[
    logging_middleware,      # पहिले लगहरू
    auth_middleware,         # त्यसपछि प्रमाणीकरण जाँच गर्दछ
    cache_middleware,        # त्यसपछि क्यास जाँच गर्दछ
    rate_limit_middleware,   # त्यसपछि दर सीमाहरू
    priority_check_middleware  # अन्त्यमा प्राथमिकता जाँच
]

# सर्तमाथी मध्यवर्ती कार्यान्वयन
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # नतिजा परिमार्जन गर्नुहोस्
    else:
        # पूर्ण रूपमा कार्यान्वयन छोड्नुहोस्
        context.result = cached_value
```

### 🔗 सम्बन्धित अवधारणाहरू:

- **एजेन्ट मिडलवेयर**: agent.run() कलहरूमा रोकथाम
- **फङ्क्शन मिडलवेयर**: टुल फङ्क्शन कलहरूमा रोकथाम (हामीले प्रयोग गरेको!)
- **मिडलवेयर पाइपलाइन**: मिडलवेयरहरूको अनुक्रममा कार्यान्वयन
- **सन्दर्भ प्रवाह**: मिडलवेयर शृङ्खलामा अवस्था पारित गर्नुहोस्


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी शुद्धताका लागि प्रयासरत छौं, तर कृपया बुझ्नुहोस् कि स्वतः अनुवादमा त्रुटि वा अशुद्धता हुन सक्छ। मूल दस्तावेज़ आफ्नो मूल भाषामा नै अधिकारिक स्रोत मान्नुपर्दछ। महत्वपूर्ण जानकारीका लागि व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार छैनौं।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
